# ADAPTIC materials

This notebook illustrates how to incorporate __ADAPTIC__ material classes from _material.py_ module.

## con1

```python
materials.con1(ID, fc1, length, fc2_factor = 0.05, characteristic = True)
```

|Parameter|Type|Description|
|:-:|:-:|:-:|
|ID|str|name of the material|
|fc1|float|peak compressive strength|
|fcm|float|mean compressive strength|
|length|float|element length|
|fc2_factor|float in range [0,1]|residual strength factor|

_Con1_ is a trilinear curve model in compression with an optional quadratic initial response . Tensile stage is given by a bilinear curve with softening.

![image](../assets/images/materials.con1_graph.png)

Initial compressive response is defined by the parameter $\alpha$, which is based on $E_{c0}$ and $E_{c1}$. Elastic initial modulus $E_{c0}$ is based on the parabolic curve. $E_{c1}$ is the secant modulus from the origin to the peak compressive stress. If $\alpha > 0$, a quadratic initial compressive response is implied.

Mean strength $f_{cm}$ is used to calculate fracture energy $G_f$. Element length $h$ is used to determine crack-band width. User can specify residual strength as the fraction of the peak strength. This is taken as 5% by default $f_{c2} = 0.05f_{c1}$.

Most of the other parameters are calculated according to <em>CEB-FIP Model Code 1990 (MC 1990)</em>, <em>CEB-FIP Model Code 2010 (MC 2010)</em> as well as <em>Rijkswaterstaat Technical Document: Guidelines for Nonlinear Finite Element Analysis of Concrete Structures (RTD 2010)</em>. These formulas are based on the uniaxial compressive cylinder strength.

<!-- To avoid overestimating the cracking moment, tensile strength $f_t$ can be reduced using tensile reduction factor *ft_factor*. Tension strain at failure $\varepsilon_{t2}$ needs to be defined by the user, taken as 0.001 by default. -->

|Parameter|Formula|Units|Reference|
|:-:|:-:|:-:|:-:|
|Compressive cylinder strength |$$f_{c} = 0.85f_{c,cube}$$|MPa|NA|
|Characteristic compressive cylinder strength |$$f_{ck}$$|MPa|NA|
|Mean compressive strength |$$f_{cm} = f_{ck} + 8$$|MPa|MC 1990 Eq. 2.1-1|
|Peak compressive strength|$$f_{c1}$$|MPa|NA|
|Residual compressive strength|$$f_{c2} = f_{c1} \cdot f_{c2,factor}$$|MPa|NA|
|Tensile strength |$$f_t= 0.3f_{cm}^{2/3} \leq C50$$ $$ f_t= 2.12ln(1+0.1f_{cm}) > C50$$|MPa|MC 2010 Eq. 5.1-3a|
|Fracture energy|$$G_f = 73\frac{ f_{cm}^{0.18}}{1000} $$|N/mm|MC 2010 Eq. 5.1-9|
|Initial compressive modulus|$$E_{c0} = 21500\cdot(f_{cm}/10)^{1/3}$$|MPa|MC 2010 Eq. 5.1-21|
|Poisson's ratio|$$0.2$$|-|MC 2010 5.1.7.3|
|Compressive fracture energy |$$G_{c} = 250G_{f}$$|N/mm|RTD 2010 p. 11|
|Compressive strain at peak strength|$$\varepsilon_{c1} = \frac{5}{3}\frac{f_{c1}}{E_0}$$|-|RTD 2010 p. 21|
|Secant compressive modulus|$$E_{c1} =  \frac{f_{c1}}{\varepsilon_{c1}}$$|MPa|NA|
|Initial tensile modulus|$$E_{t1} = E_{c0}$$|MPa|NA|
|Compressive failure strain|$$\varepsilon_{c2} = \varepsilon_{c1} + \frac{3}{2}\frac{G_c}{hf_{c1}}$$|-|RTD 2010 p. 21|
|Tensile strain at peak strength|$$\varepsilon_{t1} = \frac{f_t}{E_{t1}}$$|-|NA|
|Tensile failure strain|$$\varepsilon_{t2}=\frac{G_{f}}{h_{eq}f_{t}}$$|-|RTD 2010 p. 19|
|Initial compressive response factor|$$\alpha = \frac{E_{c0}-E_{c1}}{E_{c1}}$$|-|ADAPTIC manual|


In [1]:
import sys
sys.path.insert(1, '../libraries')
import numpy as np
import pandas as pd
import utils
import materials as mat

Below properties for _con1_ material of concrete grade C50/60 are shown. Characteristic cylinder strength is assumed $f_{c1} = f_{ck} = 50 MPa$ and element length $h = 250 mm$.

In [2]:
# Sample properties for mat_con11 instance
sample_con1 = mat.con1('C50/60', fc1 = 50, fcm = 50+8, length = 250)
sample_con1.data_frame()

,ID,$$h[mm]$$,$$f_{c1}[MPa]$$,$$f_{c2}[MPa]$$,$$f_{t}[MPa]$$,$$E_{c0}[MPa]$$,$$E_{c1}[MPa]$$,$$E_{c2}[MPa]$$,$$E_{t1}[MPa]$$,$$E_{t2}[MPa]$$,$$G_{f}[N/mm]$$,$$G_{c}[N/mm]$$,$$e_{c1}$$,$$e_{c2}$$,$$e_{t1}$$,$$e_{t2}$$,$$alpha_{c}$$
C50/60,C50/60,250,50,2.5,4.49510579008738,38629,23177.399999999998,-35375.792473868045,38629,-19527.330254932956,0.1516142070771739,37.90355176929348,0.0021572738961229475,0.0035,0.00011636609257519946,0.00034656170720366797,-0.6666666666666669


## stl1

```python
materials.stl1(ID, E1, fy, fu, epsilon_u)
```

|Parameter|Type|Description|
|:-:|:-:|:-:|
|ID|str|name of the material|
|E1|float|initial elastic stiffness|
|fy|float|yield strength|
|fu|float|ultimate strength|
|epsilon_u|float|ultimate strain|

_Stl1_ is a bilinear elasto-plastic model with kinematic strain hardening, used for a uniaxial modelling of mild steel. Although **ADAPTIC** in the current revision does not require ultimate strains, $\varepsilon_{u}$ needs to be defined for this class to be used in post-processing.

<img src="../assets/images/materials.stl1.png" width="500" />

Below properties for _stl1_ material of steel S355 are shown.

In [3]:
# Sample properties for mat_stl1 instance
sample_stl1 = mat.stl1('S355', 205000, 355, 490, 0.1)
sample_stl1.data_frame()

,ID,$$E_{1}[MPa]$$,$$E_{2}[MPa]$$,$$f_{y}[MPa]$$,$$f_{u}[MPa]$$,$$e_{y}$$,$$e_{u}$$,$$mu$$
S355,S355,205000,1373.3,355,490,0.0017,0.1,0.006699
